In [1]:
# Using one month's worth of world news posts
'''12-1-2019 EDA'''
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer converts a collection of text documents to a matrix of token counts
import warnings
from sklearn.decomposition import LatentDirichletAllocation as LDA # Load LDA model from SKL
from collections import Counter
import pyLDAvis # For interactive topic model visualization
from pyLDAvis import sklearn as sklearn_lda

C:\Users\Hannah\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [2]:
df = pd.read_csv('C:\\Users\\Hannah\\Desktop\\UT_Austin\\Analytics_MSBA\\Fall_19\\Data_mgmt_intro\\Final_project\\worldnews_0619.csv')

In [3]:
df.head(3)

,body,author,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit
0,Boarders huh,J_Man2743,1559518235,t5_2qh13,t3_a78kyo,t3_a78kyo,0,1568784725,0,0,epuwwc7,worldnews
1,[removed],[deleted],1561011745,t5_2qh13,t3_aeu81p,t1_erl6n3x,0,1570329691,0,0,erm9r84,worldnews
2,I think you know less when you think that Stal...,steeritupld,1560168854,t5_2qh13,t3_andfac,t1_eqmsv60,0,1569348202,0,0,eqmx6le,worldnews


In [4]:
# Clean data

# Remove null comments
df = df[pd.notnull(df['body'])]

# Convert all posts to string
df['body'] = df['body'].astype(str)

# Remove duplicates
df = df.drop_duplicates()

In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
# Clean data

# Remove stopwords
df['body_sw'] = df['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Remove special characters, make all characters lowercase
df['body_sw_p'] = df['body_sw'].str.replace('[^\w\s]', '').str.lower()

# Remove other common words
mask = df['body_sw_p'].str.contains('deleted','people')
df=df[~mask]

<>:7: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<ipython-input-7-26edbc285c0a>:7: DeprecationWarning: invalid escape sequence \w
  df['body_sw_p'] = df['body_sw'].str.replace('[^\w\s]', '').str.lower()


In [8]:
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the processed titles to get matrix of token counts
count_data_tokens = count_vectorizer.fit_transform(df['body_sw_p'])

In [9]:
warnings.simplefilter("ignore", DeprecationWarning)

# Function to print topics and the top 10 labels in each
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names() #words in corpus
    for topic_idx, topic in enumerate(model.components_): #enumerate adds a counter to an iterable
        print("\nTopic #%d:" % topic_idx) #%d is used as a placeholder for numeric or decimal values
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) #numpy.argsort() returns the indices that would sort an array

In [10]:
# Tweak the two parameters below
number_topics = 20
number_words = 10

# Create and fit the LDA model
# n_jobs specifyies the maximum number of concurrently running jobs. If set to -1, all CPUs are used. 
# n_components number of features which a transformer should transform the input into
lda = LDA(n_components=number_topics, n_jobs=-1, random_state=47)
lda.fit(count_data_tokens)

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
israel law state government international saudi deal gt land peace

Topic #1:
im like sure time ive pretty know thanks people really

Topic #2:
people like children want think dont know care life kids

Topic #3:
people women men like religious religion different culture society think

Topic #4:
war iran military iraq like country wars weapons think world

Topic #5:
countries uk people country eu world europe economy rich money

Topic #6:
change free speech rights human right freedom climate people like

Topic #7:
gt said far welcome version best comments reduced make tldr

Topic #8:
reddit account automatically questions contact action moderators read concerns rworldnews

Topic #9:
china chinese people hong government kong flag like protest hk

Topic #10:
people like im said think thats point know saying say

Topic #11:
money pay companies people company business tax like make government

Topic #12:
like money good food meat eat make people animals soun

In [11]:
# # Support visual to see the separation between topic components and the label frequencies for each.
# LDAvis_prepped = sklearn_lda.prepare(lda, count_data_tokens, count_vectorizer)
# pyLDAvis.display(LDAvis_prepped)
# pyLDAvis.save_html(LDAvis_prepped, './LDAvis_prepped'+ str(number_topics) +'.html')

C:\Users\Hannah\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [13]:
# # (i) A file showing which words load on which topics

# temp = pd.DataFrame(LDAvis_prepped[2])
# temp = temp.reset_index()
# temp = temp
# #.rename(columns={4: 'Wildlife', 2: 'Water', 3: 'Photography', 1: 'Landscape', 5: 'Culture'})
# pd.crosstab(temp['Term'],temp['Topic'],values=temp['Freq'],aggfunc=sum).head()

Topic,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Term,,,,,,,,,,,,,,,,,,,,
10,0.004636,0.009437,0.005794,0.097844,0.007781,0.136418,0.071686,0.090559,0.005298,0.006953,0.041389,NaN,0.030959,0.047018,0.002649,0.003973,NaN,NaN,0.39899,0.03874
101httpwwwredditcomrhelpcomments2bx3cjreddit_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999662,NaN,NaN
12nm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.975803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.970884,NaN,NaN,NaN,0.027063,NaN,NaN,NaN,NaN,NaN,NaN
